In [44]:
import pandas as pd
import numpy as np

MODEL_TYPE = "CAUSAL"

# Data

In [45]:
df = pd.read_csv(f"../../data/processed/20231220_metrics_{MODEL_TYPE}.csv")
df["reference_tokens"] = [
    eval(t)
    for t in df["reference_tokens"]
]
df["generated_tokens"] = [
    eval(t)
    for t in df["generated_tokens"]
]
df["input_tokens"] = [
    eval(t)
    for t in df["input_tokens"]
]
df.head(2)

,input,target,sesgo_pronombre,sesgo_otro,seq2seq_document,causal_document,generation,reference_tokens,max_ref_len,generated_tokens,input_tokens,bleu_gen,bleu_input,bleu_dif,rouge
0,Estimada comunidad beauchefiana: ¿Tienes papel...,['Estimada comunidad beauchefiana: ¿Tienes pap...,NO,NO,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[[Estimada, comunidad, beauchefiana, :, ¿Tiene...",11,"[Estimada, comunidad, beauchefiana, :, ¿Tienes...","[Estimada, comunidad, beauchefiana, :, ¿Tienes...",1.0,1.0,0.0,1.0
1,Desde hoy y hasta el 19 de diciembre puedes de...,['Desde hoy y hasta el 19 de diciembre puedes ...,NaN,NaN,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[[Desde, hoy, y, hasta, el, 19, de, diciembre,...",17,"[Desde, hoy, y, hasta, el, 19, de, diciembre, ...","[Desde, hoy, y, hasta, el, 19, de, diciembre, ...",1.0,1.0,0.0,1.0


In [46]:
df_with_bias = df[
    (df["sesgo_pronombre"] == "SI") | (df["sesgo_otro"] == "SI") 
]

df_without_bias = df[
    (df["sesgo_pronombre"] == "NO") & (df["sesgo_otro"] == "NO") 
]

df_not_able_to_bias = df[
    (df["sesgo_pronombre"].isna()) & (df["sesgo_otro"].isna()) 
]

print(df.shape, df_with_bias.shape[0] + df_without_bias.shape[0] + df_not_able_to_bias.shape[0])

(782, 15) 782


In [61]:
df_with_bias.shape[0], df_without_bias.shape[0], df_not_able_to_bias.shape[0]

(58, 271, 453)

# Revision

## Has bias

### No diff

In [47]:
wb_0 = df_with_bias[df_with_bias["bleu_dif"] == 0]
print(wb_0.shape)
wb_0["bleu_input"].describe()

(27, 15)


count    27.000000
mean      0.938094
std       0.064455
min       0.759099
25%       0.928111
50%       0.963071
75%       0.982439
max       0.993174
Name: bleu_input, dtype: float64

### Input more similar than generation

In [48]:
wb_below_0 = df_with_bias[df_with_bias["bleu_dif"] < 0]
wb_below_0.shape

(9, 15)

In [49]:
for i, r in wb_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

11
I: Estimados estudiantes de Pregrado , Junto con saludar les invitamos al OPEN MDS , charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM .
G: Estimada Comunidad de Pregrado , Junto con saludar les invitamos al OPEN MDS , charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM .
{'Estimados', 'Estimada', 'Comunidad', 'estudiantes'}

62
I: Cualquier duda que tengas , nos pueden mandar un mensaje 📩 por nuestro Instagram @ redes.beauchef Áreas a las que puedes postular 📑 : EstudiosLogísticaFeria LaboralExtensiónClub de ConsultoríaFinanzas✨Mucho éxito a todos quienes postulen este año✨ Redes Beauchef
G: Cualquier duda que tengas , nos pueden mandar un mensaje 📩 por nuestro Instagram @ redes.beauchef Áreas a las que puedes postular 📑 : EstudiosLogísticaFeria LaboralExtensiónClub de Consulto

- 11: Mitigacion correcta, no cubierta en el conjunto de prueba
- 62: Mitigacion correcta, no cubierta en el conjunto de prueba
- 71: Mitigacion correcta, no cubierta en el conjunto de prueba
- 184: No realiza la mitigacion esperada (competidores). Realiza una mitigacion incorrecta (algunos/as, no hace referencia a personas)
- 207: Realiza una correccion innecesaria ("los y las" por "las y los"). No realiza la mitigacion esperada (candidatos). Realiza mitigacion correcta para Consejeros (Consejeros/as). Realiza una mitigacion incorrecta (Estudiantil/a)
- 230: Cambio de palabra mal escrita (espesandolas/espesandolos). No realiza mitigacion esperada (Atentos). Realiza mitigacion no cubierta en el conjunto de prueba (los asistentes/las personas que entren). No completa con toda la secuencia entregada.
- 258: Realiza mitigacion cambiando contenido (omite 20 primeras personas)
- 340: No realiza la mitigacion esperada (candidatos). Corrige ortografia en ELECCIONE por ELECCION
- 659: Mitigacion correcta, no cubierta en el conjunto de prueba

De los 11 casos, se considera que en 4 se hizo una mitigacion correcta.

### Generation more similar than input

In [50]:
wb_above_0 = df_with_bias[df_with_bias["bleu_dif"] > 0]
print(wb_above_0.shape)
wb_above_0["bleu_gen"].describe()

(22, 15)


count    22.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: bleu_gen, dtype: float64

In [51]:
not_1 = wb_above_0[wb_above_0["bleu_gen"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

## Doesn't have bias

### No diff

In [52]:
nb_0 = df_without_bias[df_without_bias["bleu_dif"] == 0]
print(nb_0.shape)
nb_0["bleu_input"].describe()

(253, 15)


count    253.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: bleu_input, dtype: float64

In [53]:
not_1 = nb_0[nb_0["bleu_input"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

### Input more similar than generation

In [54]:
nb_below_0 = df_without_bias[df_without_bias["bleu_dif"] < 0]
nb_below_0.shape

(18, 15)

In [55]:
for i, r in nb_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

48
I: ¡Se parte de esta emocionante experiencia teatral de integración universitaria , extensión y abiertas a todo público !
G: ¡Se parte de esta emocionante experiencia teatral de integracion universitaria , extensión y abiertas a todo público !
{'integracion', 'integración'}

98
I: Los y las invitamos a revisar el Instructivo de Semestre de Verano 2023-2024 que contiene todos los detalles sobre su funcionamiento .
G: Les invitamos a revisar el Instructivo de Semestre de Verano 2023-2024 que contiene todos los detalles sobre su funcionamiento .
{'las', 'Los', 'Les', 'y'}

169
I: Luego de una larga enfermedad , partió durante las últimas horas de esta jornada , por lo que en nombre de la comunidad beauchefiana compartimos con sus seres queridos , compañeros , compañeras y profesores/as los sentimientos de pesar por esta irreparable pérdida .
G: Luego de una larga enfermedad , partió durante las últimas horas de esta jornada , por lo que en nombre de la comunidad beauchefiana compartimo

- 48: Cambio de ortografia/gramatica
- 98: Mitigacion de sesgo innecesaria
- 169: Agrega texto
- 192: Cambio de ortografia/gramatica
- 206: Cambio de ortografia/gramatica
- 255: Mitigacion de sesgo innecesaria
- 274: Cambio de ortografia/gramatica
- 315: Elimina texto
- 326: Cambio de mayuscula/minuscula
- 327: Elimina texto
- 338: Mitigacion de sesgo incorrecta
- 348: Cambio de ortografia/gramatica
- 380: Cambio de ortografia/gramatica
- 439: Cambio de mayuscula/minuscula
- 591: Mitigacion de sesgo innecesaria
- 650: Mitigacion de sesgo innecesaria
- 655: Mitigacion de sesgo incorrecta
- 736: Elimina texto

Hay 4 casos que se podria considerar correctos (mitigacion de sesgo innecesaria), y el resto son incorrectos

### Generation more similar than input

In [56]:
nb_above_0 = df_without_bias[df_without_bias["bleu_dif"] > 0]
print(nb_above_0.shape)
nb_above_0["bleu_gen"].describe()

(0, 15)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bleu_gen, dtype: float64

## Unable to bias

### No diff

In [57]:
ub_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] == 0]
print(ub_0.shape)
ub_0["bleu_input"].describe()

(440, 15)


count    440.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: bleu_input, dtype: float64

In [58]:
not_1 = ub_0[ub_0["bleu_input"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

### Input more similar than generation

In [59]:
ub_below_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] < 0]
ub_below_0.shape

(13, 15)

In [60]:
for i, r in ub_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

45
I: Saludos , Subdirección de Gestión Docente Escuela de Ingeniería y Ciencias
G: Saludos , Subdirección de Gestión Docente Escuela de Ingeniería y Ciencias ( assistant ) : Sal
{'Sal', 'assistant', '(', ')', ':'}

157
I: Inscripciones : https : //docs.google.com/forms/d/e/1FAIpQLSeoH95qz0rYLsxdKE_hIDwzPRq92RdUDvogtKslxgHiTYIclQ/viewform ? fbclid=IwAR1jFrg2wisisGOFyck5OdxS706nWyyHvhgWTjZHOe1vaw6M86BGwlmAdFkTransmisión online vía YouTube : https : //l.facebook.com/l.php ? u=https % 3A % 2F % 2 ... aTwTwpU9NaMuaFAb51v8vHbNl4xPH5G0VvsJrYs50VRhNR-Zb
G: Inscripciones : https : //docs.google.com/forms/d/e/1FAIpQLSeoH95qz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIzqz0rLYzxgHIz
{'fbclid=IwAR1jFrg2wisisGOFyck5OdxS706nWyyHvhgWTjZHOe1vaw6M86BGwlmAdFkTransmisión', '%', 'YouTube', '2F', '//docs.google.com/forms/d/e/1FAIpQLSeoH95qz

- 45: Generacion token especial en forma incorrecta
- 157: Generacion diferente a entrada
- 222: Cambio de ortografia/gramatica
- 224: Mitigacion de sesgo incorrecta
- 300: Agrega texto
- 316: Cambio de ortografia/gramatica
- 344: Agrega texto
- 485: Elimina texto
- 533: Generacion token especial en forma incorrecta
- 581: Elimina texto
- 611: Elimina texto
- 633: Cambio de ortografia/gramatica
- 639: Cambio mayuscula/minuscula

### Generation more similar than input

In [43]:
ub_above_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] > 0]
print(ub_above_0.shape)
ub_above_0["bleu_gen"].describe()

(0, 15)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bleu_gen, dtype: float64